In [ ]:
# importing 4 CSV files containing information about user history on an application named "Yammer"
# First we examine and clean the data
# Then recreate the user engagement analysis
# Lastly conduct some additional analyses to narrow down the cause

In [ ]:
import numpy as np
import scipy.stats
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_notebook()

In [ ]:
from IPython import display
from ipywidgets import interact, widgets

In [ ]:
#import the 4 files containing user and engagementment data

In [ ]:
users = pd.read_csv("users.csv")
rollup = pd.read_csv("rollup.csv")
emails = pd.read_csv("emails.csv")
events = pd.read_csv("events.csv")

In [ ]:
# take a quick look at the columns

In [ ]:
print(users.head(3))

In [ ]:
print(rollup.head(3))

In [ ]:
print(emails.head(3))

In [ ]:
print(events.head(3))

In [ ]:
# Double check the user_id is unique in users table

In [ ]:
print(len(users['user_id']), users.user_id.nunique())

In [ ]:
# merge 3 of the files on the user_id

In [ ]:
info3 = pd.merge(users, emails, on = 'user_id')

In [ ]:
info3 = pd.merge(info3, events, on = 'user_id')

In [ ]:
# take a look at the combined table

In [ ]:
print(info3.head())

In [18]:
# change the column names to keep track of which time is related to event or action

In [19]:
info3 = info3.rename(columns = {"user_id" : "user", "created_at" : "userCreated", "language" : "lang", "activated_at" : "userActivation", "state" : "state", "occurred_at_x" : "actionTime", "action" : "action", "user_type_x" : "actionUserType", "occurred_at_y" : "eventTime", "event_type" : "eventType", "event_name" : "eventName", "location" : "loc", "device" : "device", "user_type_y" : "userTypeEvent" })



In [20]:
# look at the index & info for a clean merge and get an idea of the size

In [21]:
info3.index

Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            6270657, 6270658, 6270659, 6270660, 6270661, 6270662, 6270663,
            6270664, 6270665, 6270666],
           dtype='int64', length=6270667)

In [22]:
info3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6270667 entries, 0 to 6270666
Data columns (total 15 columns):
user              float64
userCreated       object
company_id        float64
lang              object
userActivation    object
state             object
actionTime        object
action            object
actionUserType    float64
eventTime         object
eventType         object
eventName         object
loc               object
device            object
userTypeEvent     float64
dtypes: float64(4), object(11)
memory usage: 765.5+ MB


In [23]:
# using this new merged file to recreate the loss in user enagement to start
# this graph is the one that brought attention to management and what is being researched

In [1]:
info3 = info3[(info3['eventType'].str.match('engagement')) & (info3['eventName'].str.match('login'))]
info3["eventTime"] = pd.to_datetime(info3["eventTime"]).dt.week

#group by date and count unique id's
df2 = info3.groupby(['eventTime'])['user'].nunique()

#flatten the file
df2 = df2.reset_index()

#sort by date
df3 = df2.sort_values(by=['eventTime'], ascending=False)


#display
#fig = px.line(df3, x='eventTime', y='user', labels={'y':'#users'})
#fig.show()

df3.plot_bokeh.line(
    figsize=(800,450),
    y="company_id",
    title="Company use over time",
    xlabel="week",
    ylabel="No. Users Logged",
    yticks=[0,500,1000,1500,2000],
    ylim=(0,2000),
    toolbar_locations=None,
    colormap=["red","green","blue"],
    )

NameError: name 'info3' is not defined

In [ ]:
print(info3.head(1))

   user          userCreated  company_id    lang       userActivation   state  \
0   4.0  2013-01-01 14:37:51      5110.0  indian  2013-01-01 14:39:05  active   

            actionTime              action  actionUserType  eventTime  \
0  2014-05-06 09:30:00  sent_weekly_digest             3.0         20   

    eventType eventName    loc           device  userTypeEvent  
0  engagement     login  India  lenovo thinkpad            3.0  


In [ ]:
import plotly.express as px
info3 = info3[(info3['eventType'].str.match('engagement')) & (info3['eventName'].str.match('login'))]
info3["eventTime"] = pd.to_datetime(info3["eventTime"]).dt.week


#count unique user id's for each company every week
df2 = info3.groupby(['eventTime', 'company_id'])['user'].nunique()

#flatten the file
df2 = df2.reset_index()

#sort by date
df = df2.sort_values(by=['eventTime'], ascending=False)

#display
fig = px.line(df, x='eventTime', y='user', color = 'company_id',labels={'y':'#users'})
fig.show()